# Blender Test - BALANCED Lighting

Verified locally: brightness 183/255, 75% mid-tones

In [ ]:
!apt-get update -qq
!apt-get install -y -qq blender
!blender --version

In [ ]:
# Balanced lighting script
render_script = '''import bpy
import math

bpy.ops.wm.read_factory_settings(use_empty=True)

# Yellow cube (no emission)
bpy.ops.mesh.primitive_cube_add(location=(0, 0, 0), size=2)
cube = bpy.context.object

mat = bpy.data.materials.new(name="Yellow")
mat.use_nodes = True
bsdf = mat.node_tree.nodes.get("Principled BSDF")
if bsdf:
    bsdf.inputs["Base Color"].default_value = (1.0, 0.95, 0.3, 1.0)
    bsdf.inputs["Roughness"].default_value = 0.5

cube.data.materials.append(mat)

# Camera
bpy.ops.object.camera_add(location=(4, -4, 3))
camera = bpy.context.object
direction = camera.location - cube.location
rot_quat = direction.to_track_quat("-Z", "Y")
camera.rotation_euler = rot_quat.to_euler()
bpy.context.scene.camera = camera

# Moderate Sun
bpy.ops.object.light_add(type="SUN", location=(5, -3, 8))
sun = bpy.context.object
sun.data.energy = 2.0

# Fill light
bpy.ops.object.light_add(type="AREA", location=(-3, 3, 4))
area = bpy.context.object
area.data.energy = 100
area.data.size = 5

# Moderate world
scene = bpy.context.scene
if not scene.world:
    world = bpy.data.worlds.new("World")
    scene.world = world
else:
    world = scene.world

world.use_nodes = True
bg = world.node_tree.nodes.get("Background")
if bg:
    bg.inputs["Color"].default_value = (0.8, 0.8, 0.8, 1.0)
    bg.inputs["Strength"].default_value = 0.5

# CYCLES - disable denoiser
scene.render.resolution_x = 1080
scene.render.resolution_y = 1920
scene.render.engine = "CYCLES"
scene.cycles.samples = 64
scene.cycles.device = "CPU"
scene.cycles.use_denoising = False
scene.render.filepath = "/kaggle/working/test_render.png"

print(f"Rendering {scene.render.resolution_x}x{scene.render.resolution_y}...")
bpy.ops.render.render(write_still=True)
print(f"✓ Done: {scene.render.filepath}")
'''

with open('/tmp/test_render.py', 'w') as f:
    f.write(render_script)
    
print("✓ Script created (balanced lighting, NO denoiser)")

In [ ]:
!blender --background --python /tmp/test_render.py

In [ ]:
import os
from PIL import Image
import numpy as np
from IPython.display import display

if os.path.exists('/kaggle/working/test_render.png'):
    img = Image.open('/kaggle/working/test_render.png')
    arr = np.array(img)
    mean = arr.mean()
    
    print(f"✓ SUCCESS: {img.size}")
    print(f"Brightness: {mean:.1f}/255")
    
    if mean > 200:
        print("⚠️ Possibly overexposed")
    elif mean < 30:
        print("⚠️ Very dark")  
    else:
        print("✅ Balanced range")
    
    display(img)
else:
    print("✗ FAILED")
    !ls -lh /kaggle/working/